In [1]:
import pandas as pd
import re
from urllib.request import urlopen

In [2]:
# find II subjects
url = 'https://www.uib.no/ii/emner' 
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")

subjects = re.findall('<a href="/nb/emne/(.*?)">', html)
subject_urls = {subject: 'https://www.uib.no/nb/emne/' + subject for subject in subjects}

In [3]:
def extract_subject_information(emnekode, url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")

    tittel = re.findall('<h1 class="course-title">(.*?)</h1>', html)[0]
    nivå = re.findall('<h2 class="course-level">(.*?)</h2>', html)[0]   
    
    try:
        semester = re.findall('<dt>Undervisningssemester</dt>\n(.*?)\n', html)[0] 
        semester = re.sub("</?dd[^>]*>", "", semester.strip())
    except IndexError:
        semester = "Uvisst" 
    
    studiepoeng = float(re.findall('<dt>Studiepoeng</dt>\n                  <dd>(.*?)</dd>\n', html)[0])

    try: 
        vurdering = re.findall('<dt>Vurderingsordning</dt>\n        <dd>(.*?)</dd>\n', html)[0] 
    except IndexError:
        return dict(emnekode=emnekode, 
                tittel=tittel, 
                semester=semester, 
                nivå=nivå, 
                studiepoeng=studiepoeng)
        

    return dict(emnekode=emnekode, 
                tittel=tittel, 
                semester=semester, 
                nivå=nivå, 
                studiepoeng=studiepoeng,
                vurdering=vurdering)

In [4]:
df = pd.DataFrame([extract_subject_information(emnekode, url) for emnekode, url in subject_urls.items()])

KeyboardInterrupt: 

In [ ]:
df.to_csv('informatikkemner_uib.csv', index=False)